Whole Slide Images (WSI) are captured at a very high resolution. In this dataset, 21 anonymised WSI images in Aperio SVS format, of canine carcinoma, are available. The cell annotations are provided in a SQLite3 database. 

Below we extract the dimensions of all the 21 images from the database. 

In [1]:
from SlideRunner.dataAccess.database import Database
import numpy as np

def listOfSlides(DB):
    DB.execute('SELECT uid, filename, width, height from Slides')
    return DB.fetchall()

DB = Database()
DB.open('databases/MITOS_WSI_CMC_COADEL_TR.sqlite')

for slideid,filename, width, height in listOfSlides(DB):
    print(f"Slide ID# {slideid}: {filename} {height} x {width} pixels")


Slide ID# 1: a8773be388e12df89edd.svs 57462 x 61199 pixels
Slide ID# 2: 460906c0b1fe17ea5354.svs 63831 x 136800 pixels
Slide ID# 3: 2d56d1902ca533a5b509.svs 65453 x 70199 pixels
Slide ID# 5: d0423ef9a648bb66a763.svs 85208 x 116999 pixels
Slide ID# 6: 50cf88e9a33df0c0c8f9.svs 87273 x 125999 pixels
Slide ID# 7: da18e7b9846e9d38034c.svs 84305 x 95399 pixels
Slide ID# 8: d7a8af121d7d4f3fbf01.svs 88592 x 111599 pixels
Slide ID# 9: 2191a7aa287ce1d5dbc0.svs 82534 x 98999 pixels
Slide ID# 10: 69a02453620ade0edefd.svs 90076 x 97199 pixels
Slide ID# 11: c4b95da36e32993289cb.svs 84710 x 124199 pixels
Slide ID# 12: fa4959e484beec77543b.svs 84406 x 118799 pixels
Slide ID# 13: 72c93e042d0171a61012.svs 78623 x 122399 pixels
Slide ID# 14: 4eee7b944ad5e46c60ce.svs 47721 x 44999 pixels
Slide ID# 15: b1bdee8e5e3372174619.svs 83745 x 125999 pixels
Slide ID# 16: 3d3d04eca056556b0b26.svs 87152 x 109799 pixels
Slide ID# 17: 084383c18b9060880e82.svs 82153 x 61199 pixels
Slide ID# 18: e09512d530d933e436d5.svs 

/opt/homebrew/Caskroom/miniforge/base/envs/MITOS_WSI_CMC/lib/python3.9/site-packages/SlideRunner-2.2.0-py3.9.egg/SlideRunner/dataAccess/database.py:2: UserWarning: Deprecated package. Please use the SlideRunner_dataAccess package, and not the SlideRunner main package for data access.
  warnings.warn('Deprecated package. Please use the SlideRunner_dataAccess package, and not the SlideRunner main package for data access.')


Let us load and display a WSI image with some sample annotations.

In [98]:
def get_annotation_classes(DB, slide):
    level = 0 #slide.level_count - 1
    level_dimension = slide.level_dimensions[level]
    down_factor = slide.level_downsamples[level]

    labels, bboxes = [], []
    annotations = dict()
    positive_class = 1
    negative_class = 7
    classes = {positive_class: 1}

    for id, annotation in DB.annotations.items():
        if annotation.deleted or annotation.annotationType != AnnotationType.SPOT:
            continue
    
        annotation.r = 25
        d = 2 * annotation.r / down_factor
        x_min = (annotation.x1 - annotation.r) / down_factor
        y_min = (annotation.y1 - annotation.r) / down_factor
        x_max = x_min + d
        y_max = y_min + d
        if annotation.agreedClass not in annotations:
            annotations[annotation.agreedClass] = dict()
            annotations[annotation.agreedClass]['bboxes'] = list()
            annotations[annotation.agreedClass]['label'] = list()
    
        annotations[annotation.agreedClass]['bboxes'].append([int(x_min), int(y_min), int(x_max), int(y_max)])
        annotations[annotation.agreedClass]['label'].append(annotation.agreedClass)
    
        if annotation.agreedClass in classes:
            label = classes[annotation.agreedClass]
            bboxes.append([int(x_min), int(y_min), int(x_max), int(y_max)])
            labels.append(label)
        
    return annotations


In [100]:
import openslide
import pandas as pd

for slideid,filename, width, height in listOfSlides(DB):
    DB.loadIntoMemory(slideid)
    total_annotations = len(DB.annotations.keys())
    print(f"{filename} has {total_annotations} annotations")
    annoList = [[anno.x1, anno.y1, anno.r] for anno in DB.annotations.values() 
                if not anno.deleted and anno.annotationType == AnnotationType.SPOT]
    print(f"Total annotations = {total_annotations}; Spot annotations = {len(annoList)}")

    df = pd.DataFrame(annoList, columns = ['x', 'y', 'r'])
    print(df['r'].describe())
    
    slide = openslide.open_slide('WSI/' + filename)
    annotation_classes = get_annotation_classes(DB, slide)
    

    print(f"#Annotations for class 1: {len(annotation_classes[1]['bboxes'])}")

    print(f"#Annotations for class 2: {len(annotation_classes[2]['bboxes'])}")

a8773be388e12df89edd.svs has 667 annotations
Total annotations = 667; Spot annotations = 662
count    662.0
mean      25.0
std        0.0
min       25.0
25%       25.0
50%       25.0
75%       25.0
max       25.0
Name: r, dtype: float64
#Annotations for class 1: 590
#Annotations for class 2: 72
460906c0b1fe17ea5354.svs has 3223 annotations
Total annotations = 3223; Spot annotations = 3222
count    3222.0
mean       25.0
std         0.0
min        25.0
25%        25.0
50%        25.0
75%        25.0
max        25.0
Name: r, dtype: float64
#Annotations for class 1: 2474
#Annotations for class 2: 748
2d56d1902ca533a5b509.svs has 522 annotations
Total annotations = 522; Spot annotations = 519
count    519.0
mean      25.0
std        0.0
min       25.0
25%       25.0
50%       25.0
75%       25.0
max       25.0
Name: r, dtype: float64
#Annotations for class 1: 364
#Annotations for class 2: 155
d0423ef9a648bb66a763.svs has 2009 annotations
Total annotations = 2009; Spot annotations = 2007
co